In [ ]:
# Mount to my Google Drive
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/MyDrive/###")

In [ ]:
# Check what GPU we are using
!nvidia-smi -L

In [ ]:
# Import necessary packages
import tensorflow as tf
import numpy as np
import pickle
import cvxpy as cvx
import matplotlib.pyplot as plt

In [ ]:
# Import our code
from linear_regression_generalization import generate_sparse_dataset, train_linreg_network

# Model Setup

In [ ]:
# Dimension of problem
d = 250

# Sparsity of problem 
# There are r nonzero coordinates of the beta vector, and they are the first five coordinates
r = 5
indices = range(5)

In [ ]:
# Number of training points
N =  100

In [ ]:
# Generate our training dataset
train_data = generate_sparse_dataset(N, d, r, indices=indices, filename=f'training_data_sparse_regression_{N}.pk')

In [ ]:
# And our test dataset
num_test = 100
test_data = generate_sparse_dataset(num_test, d, r, indices=indices, filename='test_data_sparse_regression.pk')

In [ ]:
# Initialization shape and scales for our NN
w0 = tf.ones([2*d, 1])
alphas = list(np.logspace(-3, 0, 20))

In [ ]:
# Load in previous data
with open(f'training_data_sparse_regression_{N}.pk', 'rb') as f:
    train_data = pickle.load(f)

with open('test_data_sparse_regression.pk', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
# Get the minimum l^1 norm solution for the training data
# Code from Woodworth et al. 2020
beta = cvx.Variable(train_data[0].shape[1])
prob = cvx.Problem(cvx.Minimize(cvx.norm(beta,1)), [train_data[0] @ beta == np.reshape(train_data[1], [-1])])
prob.solve()
solution_l1_norm = prob.value
beta_min_l1 = beta.value
beta_min_l1 = np.reshape(beta_min_l1, [-1, 1])

# Model Training

In [ ]:
# Now let's train the models
output = train_linreg_network((train_data[0], train_data[1]), (test_data[0], test_data[1]), w0, alphas=alphas, lr=3*10**(-4))

In [ ]:
# Save the output from training for posterity
with open(f'sparse_generalization_{N}.pk', 'wb') as f:
        pickle.dump(output, f)

# Visualizing Results

In [ ]:
# Import prior outputs
with open('sparse_generalization_50.pk', 'rb') as f:
    output50 = pickle.load(f)

with open('sparse_generalization_100.pk', 'rb') as f:
    output100 = pickle.load(f)

with open('sparse_generalization_200.pk', 'rb') as f:
    output200 = pickle.load(f)

In [ ]:
# Boolean lists, True if gradient descent does converge within 10^4 iterations, False otherwise (no convergence)
converge50 = [i for i in range(len(output50[2])) if output50[2][i] < 9999]
converge100 = [i for i in range(len(output100[2])) if output100[2][i] < 9999]
converge200 = [i for i in range(len(output200[2])) if output200[2][i] < 9999]

In [ ]:
# Now plot the MSE on the test dataset against the initialization scale \alpha for each of the models we trained
# A dashed line indicates that the model did not achieve the desired convergence
fig = plt.figure(0)

MSE50 = [output50[1][i][-1] for i in range(len(alphas))]
MSE100 = [output100[1][i][-1] for i in range(len(alphas))]
MSE200 = [output200[1][i][-1] for i in range(len(alphas))]

# plt.plot(alphas[0:min(converge50)], MSE50[0:min(converge50)], color="#2ca02c", linestyle="dashed")
# plt.plot(alphas[(min(converge50)-1):], MSE100[(min(converge50)-1):], color="#2ca02c", label="50")

plt.plot(alphas[0:min(converge100)], MSE100[0:min(converge100)], color="#1f77b4", linestyle="dashed")
plt.plot(alphas[(min(converge100)-1):], MSE100[(min(converge100)-1):], color="#1f77b4", label="100")

plt.plot(alphas[0:min(converge200)], MSE200[0:min(converge200)], color="#ff7f0e", linestyle="dashed")
plt.plot(alphas[(min(converge200)-1):], MSE200[(min(converge200)-1):], color="#ff7f0e",label="200")

plt.xscale("log")
plt.yscale("log")
plt.legend()

plt.xlabel(r"Initialization Scale ($\alpha$)")
plt.ylabel("Test MSE")

fig.show()
fig.savefig('visualize_test_MSE_log.png', dpi=300)

In [ ]:
# Rather than looking at the MSE of the model on the test dataset, we can instead consider the \ell^2 distance between the learned and true 
# beta vector, as Woodworth and colleagues do in their paper. This is because, by the Riesz representation theorem, the model evaluated at any input x
# is completely detemined by \beta according to the \ell^2 inner product of \beta and x.  
fig = plt.figure(0)

beta_0 = train_data[2]

l2error50 = [np.linalg.norm((np.square(output50[3][i].numpy())[0:d] - np.square(output50[3][i].numpy())[d:]) - beta_0, ord=2)**2 for i in range(len(alphas))]
l2error100 = [np.linalg.norm((np.square(output100[3][i].numpy())[0:d] - np.square(output100[3][i].numpy())[d:]) - beta_0, ord=2)**2 for i in range(len(alphas))]
l2error200 = [np.linalg.norm((np.square(output200[3][i].numpy())[0:d] - np.square(output200[3][i].numpy())[d:]) - beta_0, ord=2)**2 for i in range(len(alphas))]

plt.plot(alphas[0:min(converge50)], l2error50[0:min(converge50)], color="#2ca02c", linestyle="dashed")
plt.plot(alphas[(min(converge50)-1):], l2error50[(min(converge50)-1):], color="#2ca02c", label="50")

plt.plot(alphas[0:min(converge100)], l2error100[0:min(converge100)], color="#1f77b4", linestyle="dashed")
plt.plot(alphas[(min(converge100)-1):], l2error100[(min(converge100)-1):], color="#1f77b4", label="100")

plt.plot(alphas[0:min(converge200)], l2error200[0:min(converge200)], color="#ff7f0e", linestyle="dashed")
plt.plot(alphas[(min(converge200)-1):], l2error200[(min(converge200)-1):], color="#ff7f0e",label="200")

plt.xscale("log")
plt.yscale("log")
plt.legend()

plt.xlabel(r"Initialization Scale ($\alpha$)")
plt.ylabel(r"$\ell^2$ Population Error  $\Vert \beta - \beta^* \Vert_2^2$")

fig.show()
fig.savefig('12_population_error_log_all.png', dpi=300)

In [ ]:
# Likewise, we look at the \ell^1 distance between the learned and true beta vector as a measure of the population error

l1error50 = [np.linalg.norm((tf.math.square(i)[0:d] - tf.math.square(i)[d:]) - beta_0, ord=1) for i in output50[3]]
l1error100 = [np.linalg.norm((tf.math.square(i)[0:d] - tf.math.square(i)[d:]) - beta_0, ord=1) for i in output100[3]]
l1error200 = [np.linalg.norm((tf.math.square(i)[0:d] - tf.math.square(i)[d:]) - beta_0, ord=1) for i in output200[3]]

fig = plt.figure(1)

plt.plot(alphas[0:min(converge50)], l1error50[0:min(converge50)], color="#2ca02c", linestyle="dashed")
plt.plot(alphas[(min(converge50)-1):], l1error50[(min(converge50)-1):], color="#2ca02c", label="50")

plt.plot(alphas[0:min(converge100)], l1error100[0:min(converge100)], color="#1f77b4", linestyle="dashed")
plt.plot(alphas[(min(converge100)-1):], l1error100[(min(converge100)-1):], color="#1f77b4", label="100")

plt.plot(alphas[0:min(converge200)], l1error200[0:min(converge200)], color="#ff7f0e", linestyle="dashed")
plt.plot(alphas[(min(converge200)-1):], l1error200[(min(converge200)-1):], color="#ff7f0e",label="200")


plt.xscale("log")
plt.yscale("log")
plt.legend()

plt.xlabel(r"Initialization Scale ($\alpha$)")
plt.ylabel(r"$\ell^1$ Population Error  $\Vert \beta - \beta_{w} \Vert_1$")
plt.show()
fig.savefig('11_population_error_log_all.png', dpi=300)

In [ ]:
# Plot the \ell^1 distance between the learned beta vector and the minimum \ell^1 solution to the system X\beta = y

l1error50 = [np.linalg.norm((tf.math.square(i)[0:d] - tf.math.square(i)[d:]) - beta_min_l1, ord=1) for i in output50[3]]
l1error100 = [np.linalg.norm((tf.math.square(i)[0:d] - tf.math.square(i)[d:]) - beta_min_l1, ord=1) for i in output100[3]]
l1error200 = [np.linalg.norm((tf.math.square(i)[0:d] - tf.math.square(i)[d:]) - beta_min_l1, ord=1) for i in output200[3]]

fig = plt.figure(1)

plt.plot(alphas[0:min(converge50)], l1error50[0:min(converge50)], color="#2ca02c", linestyle="dashed")
plt.plot(alphas[(min(converge50)-1):], l1error50[(min(converge50)-1):], color="#2ca02c", label="50")

plt.plot(alphas[0:min(converge100)], l1error100[0:min(converge100)], color="#1f77b4", linestyle="dashed")
plt.plot(alphas[(min(converge100)-1):], l1error100[(min(converge100)-1):], color="#1f77b4", label="100")

plt.plot(alphas[0:min(converge200)], l1error200[0:min(converge200)], color="#ff7f0e", linestyle="dashed")
plt.plot(alphas[(min(converge200)-1):], l1error200[(min(converge200)-1):], color="#ff7f0e",label="200")


plt.xscale("log")
plt.yscale("log")
plt.legend()

plt.xlabel(r"Initialization Scale ($\alpha$)")
plt.ylabel(r"Excess $\ell^1$ Norm  $\Vert \beta - \beta^{\ell^1} \Vert_1$")
plt.show()
fig.savefig('excess_l1_log_all.png', dpi=300)

In [ ]:
# Plot the number of iterations necessary for gradient descent to converge below 10^(-4),
# our predetmined threshold on the training loss
fig = plt.figure(2)

plt.plot(alphas, output50[2], color="#2ca02c", label="50")

plt.plot(alphas, output100[2], color="#1f77b4", label="100")

plt.plot(alphas, output200[2], color="#ff7f0e", label="200")

plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.xscale("log")
plt.legend()

plt.xlabel(r"Initialization Scale ($\alpha$)")
plt.ylabel("Number of Training Epochs")

fig.show()
fig.savefig('interations_to_convergence_all.png', dpi=300)

In [ ]:
# Plot the first 10 components of the empirical solution vector \beta against those of the "true" solution vector \beta^*
fig = plt.figure(3)

plt.plot(np.arange(1, 11), beta_0[0:10], '.', label=r"$\beta_w$")
plt.plot(np.arange(1, 11), (tf.math.square(output100[3][0])[0:d] - tf.math.square(output100[3][0])[d:])[0:10], '.', label=r"$\beta_{\alpha = 10^{-3}}$")

plt.legend()
fig.savefig('visualize_solution_vec_0.001.png', dpi=300)